# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 15__

__Nombres de los estudiantes: Javier Sande Ríos y Mario Sanz Guerrero__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [2]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,es". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

#### Solución:

In [3]:
%%wdsparql
SELECT ?fecha ?lugarLabel ?lugar
WHERE 
{
  wd:Q34981 wdt:P569 ?fecha;
            wdt:P19 ?lugar.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,fecha,lugarLabel,lugar
0,1920-01-02T00:00:00Z,Petrovichi,http://www.wikidata.org/entity/Q1010621


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

#### Solución:

In [4]:
%%wdsparql
SELECT ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?profesionLabel

,profesionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

#### Solución:

In [7]:
%%wdsparql
SELECT ?profesionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?profesion.
  ?profesion wdt:P279* wd:Q36180.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ?profesionLabel

,profesionLabel
0,autobiographer
1,biochemist
2,journalist
3,non-fiction writer
4,novelist
5,prosaist
6,science fiction writer
7,science writer
8,scientist
9,screenwriter


Nos llama la atención que se devuelvan todas las 12 profesiones que se han devuelto para la consulta anterior. Hay algunas profesiones como _autobiographer_ que es obvio que son subclase de _escritor_, pero otras que no lo son tanto como por ejemplo _biochemist_. Por ese motivo, vamos a meternos en la web de Wikidata para ver a qué nivel son subclase de _escritor_ estas profesiones que no parecen tan relacionadas (la flecha derecha representa "es subclase de"):

* Biochemist: _biochemist_ &#8594; _biologist_ &#8594; _scientist_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_
* Scientist: _scientist_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_
* University teacher: _university teacher_ &#8594; _academic_ &#8594; _scholar_ &#8594; _researcher_ &#8594; _non-fiction writer_ &#8594; _writer_

Las demás profesiones parecen bastante más relacionadas con _escritor_. Esto ha servido para demostrar que la consulta está bien hecha.

### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

#### Solución:

### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

#### Solución:

### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

#### Solución:

### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

#### Solución:

### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

#### Solución:

### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

#### Solución:

### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

#### Solución:

### Consulta 11

¿Cuántos asertos hay sobre Asimov en Wikidata? Ten en cuenta que Asimov puede aparece tanto como sujeto como objeto de cada tripleta.

#### Solución:

__Fecha de las consultas: 21 de abril de 2022__